In [ ]:
#import libraries

In [ ]:
import itertools
import multiprocessing as mp
import math
import kneed
import networkx as netx
from scipy.spatial import Delaunay
import numpy as np

In [ ]:
#creates a list filled with zeros as long as the parameter

In [ ]:
def zerolist(n):
    list_zero = [0] * n
    return list_zero

In [ ]:
# finds the nodes,adjacency,edges and number of nodes attributes of graph

In [ ]:
def findCompGraph(graph):
    nodes = dict(graph.nodes())
    adjacency = dict(graph.adjacency())
    node_num = graph.number_of_nodes()
    edges = dict(graph.edges())
    return nodes, adjacency, edges, node_num

In [ ]:
#the cutESC function that calls the other functions

In [ ]:
def cutESC(data):
    graph = DelaunayG(data)
    gabriel(graph)
    gloEdges(graph)
    findLoc(graph)
    findOtherLocal(graph)
    label, Scc = assignClusterLabels(graph)
    return label, Scc

In [ ]:
#CutESC algorithm starts here and in the code below a spatial graph is generated using Delaunay graph

In [ ]:
def DelaunayG(data):
    tri = Delaunay(data, qhull_options='QJ Pp')  # delaunay generator
    edges = []
    for k in tri.simplices:
        for indices in itertools.combinations(k, 2):
            edges.append(indices)

    graph = netx.Graph(edges)  # this is the edges(connnections)
    nodes = dict(graph.nodes())
    for i in nodes:
        nodes[i]['node'] = np.array(data[i])
    adjacency = dict(graph.adjacency())
    removed = []
    for i in nodes:
        for j in adjacency[i]:
            weight = np.linalg.norm(nodes[i]['node'] - nodes[j]['node'])
            if weight != 0:
                graph.edges[(i, j)]['weight'] = weight
            else:
                removed.append((i, j))
    graph.remove_edges_from(removed)
    return graph

In [ ]:
#the part that checks_edges for the pool function

In [ ]:
def check_edge(i):
    try:
        removed_edges = []
        for j in adjacencys[i]:
            dist_ij = np.linalg.norm(node[i]['node'] - node[j]['node'])
            for k in adjacencys[i]:
                if j == k:
                    continue
                dist_ik = np.linalg.norm(node[i]['node'] - node[k]['node'])
                dist_jk = np.linalg.norm(node[j]['node'] - node[k]['node'])
                if dist_ij ** 2 > dist_ik ** 2 + dist_jk ** 2:
                    removed_edges.append((i, j))
                    break
        return removed_edges
    except Exception as e:
        print('failed', flush=True)
        return e

In [ ]:
#gabriel graph function

In [ ]:
def gabriel(graph):
    global node
    global adjacencys
    node = dict(graph.nodes())
    adjacencys = dict(graph.adjacency())
    if __name__ == "__main__":
        pool = mp.Pool(processes=mp.cpu_count() - 1)
        removed_edges = pool.map(check_edge, node)
        removed_edges = set(itertools.chain.from_iterable(removed_edges))
        pool.close()
        graph.remove_edges_from(removed_edges)

In [ ]:
#In this part of the graph globally long edges are found and removed from the graph

In [ ]:
def gloEdges(graph):
    nodes, adjacency, edges, node_num = findCompGraph(graph)
    weights = list(netx.get_edge_attributes(graph, 'weight').values())
    mean_loc = zerolist(node_num)

    for i in nodes:  # the mean length of incident edges of vertex Pi
        loc_m = 0
        for j in adjacency[i]:
            loc_m = loc_m + graph.edges[i, j]["weight"]
        if len(adjacency[i]) == 0:
            mean_loc[i] = loc_m  # mean(Pi)
            continue
        else:
            loc_m = loc_m / len(adjacency[i])
            mean_loc[i] = loc_m  # mean(Pi)

    global_m = np.mean(weights)  # mean(GG)          #the mean length of all edges in gabriel graph
    glo_std = 0

    for i in nodes:  # the standard derivation of the mean length of edges in neigborhood
        glo_std = glo_std + math.pow(global_m - mean_loc[i], 2)
    glo_std = math.sqrt(glo_std / (node_num - 1))  # std(GG)
    GCuti = list()  # cut edge value

    for i in range(node_num):
        if mean_loc[i] == 0:
            GCuti.append(0)
        else:
            var = global_m * glo_std / mean_loc[i]
            var = var + global_m
            GCuti.append(var)

    remove = list()  # remove globally long edges

    for i in edges:
        if (edges[i]["weight"] >= GCuti[i[0]]) or (edges[i]["weight"]) >= GCuti[i[1]]:
            remove.append(i)
    graph.remove_edges_from(remove)

In [ ]:
#In this part locally long edges found and removed from the graph

In [ ]:
def findLoc(graph):
    graph_var = [var for var in sorted(
        netx.connected_components(graph), key=len, reverse=True)]
    label = zerolist(graph.number_of_nodes())
    for i, var in enumerate(graph_var):
        for j in var:
            label[j] = i
    nodes, adjacency, edges, node_num = findCompGraph(graph)
    loc_st = zerolist(node_num)
    graph_var_m = zerolist(len(graph_var))

    for i in nodes:  # the mean length of edges in the second order neighborhood of a vertex Pi in a subgraph Gx
        loc_m = 0
        nei_num = 0
        for j in adjacency[i]:
            weight = graph.edges[i, j]['weight']
            loc_m = loc_m + weight
            nei_num = nei_num + 1
            for n in adjacency[j]:
                weight2 = graph.edges[j, n]['weight']
                loc_m = loc_m + weight2
                nei_num = nei_num + 1

        if nei_num != 0:
            loc_m = loc_m / nei_num
        graph_var_m[label[i]] = loc_m + graph_var_m[label[i]]  # mean(Pi)

    for i in range(len(graph_var_m)):
        graph_var_m[i] = graph_var_m[i] / len(graph_var[i])  # mean(Gk)

    for i in nodes:  # the standard derivation of all edges that are directly connected to vertex Pi
        loc_std = 0
        nei_num = 0
        for j in adjacency[i]:
            weight = graph.edges[i, j]['weight']
            loc_std = loc_std + math.pow(graph_var_m[label[i]] - weight, 2)
            nei_num = nei_num + 1
            for n in adjacency[j]:
                weight = graph.edges[j, n]['weight']
                loc_std = loc_std + math.pow(graph_var_m[label[i]] - weight, 2)
                nei_num = nei_num + 1
        if nei_num != 0:
            loc_std = math.sqrt(loc_std / (nei_num - 1))
        loc_st[i] = loc_std  # std(Pi)

    alpha = 1.0  # should be between 0 and 1 set to 1 by default
    remove = list()  # remove part
    for i in edges:
        try:
            val1 = math.exp(graph_var_m[label[i[0]]] / graph.edges[i]['weight'])
            val1 = alpha * loc_st[i[0]] * val1
            val1 = graph_var_m[label[i[0]]] + val1
        except OverflowError:
            val1 = float('inf')
        try:
            val2 = math.exp(graph_var_m[label[i[1]]] / graph.edges[i]['weight'])
            val2 = alpha * loc_st[i[1]] * val2
            val2 = graph_var_m[label[i[1]]] + val2
        except OverflowError:
            val2 = float('inf')
        if (edges[i]['weight'] >= val1) or (edges[i]['weight'] >= val2):
            remove.append(i)
    graph.remove_edges_from(remove)

In [ ]:
# In this section other locally long edges are identified and removed

In [ ]:
def findOtherLocal(graph, beta=1.0):
    # finding other locally long edges
    nodes, adjacency, edges, node_num = findCompGraph(graph)
    loc_me = zerolist(node_num)
    loc_st = zerolist(node_num)
    for i in nodes:  # the mean length of edges in the second order neighborhood of a vertex Pi in a  new subgraph
        loc_m = 0
        nei_num = 0
        for j in adjacency[i]:
            weight = graph.edges[i, j]['weight']
            loc_m = loc_m + weight
            nei_num = nei_num + 1
            for n in adjacency[j]:
                weight = graph.edges[j, n]['weight']
                loc_m = loc_m + weight
                nei_num = nei_num + 1

        if nei_num == 0:
            nei_m = loc_m
            loc_me[i] = nei_m  # mean(Pi)
            continue
        if nei_num != 0:
            nei_m = loc_m / nei_num
            loc_me[i] = nei_m  # mean(Pi)

    for i in nodes:  # the standard derivation of all edges that are directly connected to vertex Pi
        loc_std = 0
        nei_num = 0
        for j in adjacency[i]:
            weight = graph.edges[i, j]['weight']
            loc_std = loc_std + math.pow(loc_me[i] - weight, 2)
            nei_num = nei_num + 1
            for n in adjacency[j]:
                weight = graph.edges[j, n]['weight']
                loc_std = loc_std + math.pow(loc_me[i] - weight, 2)
                nei_num = nei_num + 1

        if nei_num == 0:
            loc_st[i] = loc_std  # std(Pi)
            continue
        if nei_num != 0:
            if nei_num != 1:
                loc_std = math.sqrt(loc_std / (nei_num - 1))
            loc_st[i] = loc_std  # std(Pi)

    beta = 1  # should be between 0 and 1 set to 1 by default
    remove = list()  # remove part
    for i in edges:
        try:
            val1 = math.exp(loc_me[i[0]] / graph.edges[i]['weight'])
            val1 = beta * loc_st[i[0]] * val1
            val1 = loc_me[i[0]] + val1
        except OverflowError:
            val1 = float('inf')
        try:
            val2 = math.exp(loc_me[i[1]] / graph.edges[i]['weight'])
            val2 = beta * loc_st[i[1]] * val2
            val2 = loc_me[i[1]] + val2
        except OverflowError:
            val2 = float('inf')
        if (edges[i]['weight'] >= val1) or (edges[i]['weight'] >= val2):
            remove.append(i)
    graph.remove_edges_from(remove)


In [ ]:
#The part below assigns labels to clusters by using elbow method

In [ ]:
def assignClusterLabels(graph):
    num_node = graph.number_of_nodes()

    Scc = [var for var in sorted(
        netx.connected_components(graph), key=len, reverse=True)]
    label = zerolist(num_node)
    for i, var in enumerate(Scc):
        for j in var:
            label[j] = i # find strongly connected components in the graph and sort it by descending order


    histog = [len(var) for var in Scc]  # finding number of vertices in each Scc connected components


    lengl = len(histog)
    leng = list(range(lengl))
    knees = kneed.KneeLocator(leng, histog, S=1.0, curve="convex", direction="decreasing")
    idx = knees.knee  # locate elbow points

    for i, var in enumerate(Scc[idx:]):
        for j in var:
            label[j] = -1

    label = np.array(label)
    Scc = np.array(Scc)
    return label, Scc